In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

citations:
https://www.geeksforgeeks.org/data-analysis/exploratory-data-analysis-in-python/


In [10]:
TWEETS_PATH= "../data/raw_tweets_text.csv"
SENTIMENT_PATH="../data/t4sa_text_sentiment.tsv"


load + small transformations for more readable dataset

In [11]:
#load data
tweets_df = pd.read_csv(TWEETS_PATH, encoding='latin-1', header=0)
sentiment_df= pd.read_csv(SENTIMENT_PATH, sep='\t', header=0)


## Get Basic information

In [12]:
print(tweets_df.shape)
print(sentiment_df.shape)

(3452663, 2)
(1179957, 4)


In [13]:
print(tweets_df.info())
print(sentiment_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3452663 entries, 0 to 3452662
Data columns (total 2 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   id      int64 
 1   text    object
dtypes: int64(1), object(1)
memory usage: 52.7+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1179957 entries, 0 to 1179956
Data columns (total 4 columns):
 #   Column  Non-Null Count    Dtype  
---  ------  --------------    -----  
 0   TWID    1179957 non-null  int64  
 1   NEG     1179957 non-null  float64
 2   NEU     1179957 non-null  float64
 3   POS     1179957 non-null  float64
dtypes: float64(3), int64(1)
memory usage: 36.0 MB
None


In [14]:

# split second column of tweets into first word, main tweet, link 
# Process in chunks to avoid running out of memory
chunk_size = 10000
num_rows = tweets_df.shape[0]

main_text = []
last_link = []

for start in range(0, num_rows, chunk_size):
    end = min(start + chunk_size, num_rows)
    text_chunk = tweets_df['text'].iloc[start:end].str.split()
    last_link.extend(text_chunk.str[-1].where(text_chunk.str[-1].str.startswith('http'), ''))
    main_text.extend(text_chunk.apply(lambda x: ' '.join(x[:-1]) if x and x[-1].startswith('http') else ' '.join(x) if x else ''))

tweets_df['main_text'] = main_text
tweets_df['last_link'] = last_link



In [15]:
tweets_df.nunique()

id           3452663
text         3176206
main_text    3024375
last_link    3006894
dtype: int64

the below murdered ,my computer


In [16]:

# filter 
matching_ids = pd.Index(tweets_df['id']).intersection(sentiment_df['TWID'])

exclusive_tweets_df = tweets_df[~tweets_df['id'].isin(matching_ids)]
exclusive_sentiment_df = sentiment_df[~sentiment_df['TWID'].isin(matching_ids)]

# 
merged_df = pd.concat([
    exclusive_tweets_df.set_index('id'),
    exclusive_sentiment_df.set_index('TWID')
], axis=0, join='outer', ignore_index=False).reset_index()



# EDA section

do your eda here
you probably want to create a copy of the dataset before making any transformations(so you dont have to reload the entire notebook if you make a mistake)

In [19]:
tweets_df.head()
merged_df.head()

,index,text,main_text,last_link,NEG,NEU,POS
0,758014713804587008,RT @polarcomic: And surprise! the #RegularShow...,RT @polarcomic: And surprise! the #RegularShow...,https://t.co/kCctJpâ¦,NaN,NaN,NaN
1,758014717990428672,RT @SweetBabyBellB: My unproblematic fav who k...,RT @SweetBabyBellB: My unproblematic fav who k...,https://t.co/A9RK5b0Hfm,NaN,NaN,NaN
2,758014646716665857,RT @WhyLarryIsReal: I mean we know harry isn't...,RT @WhyLarryIsReal: I mean we know harry isn't...,https://t.co/fW2TEwSHEq,NaN,NaN,NaN
3,758014655071526912,"RT @Eastbay: She's ready, resilient, and on ou...","RT @Eastbay: She's ready, resilient, and on ou...",https://t.co/j4JwiEgCmd,NaN,NaN,NaN
4,758014642526429184,RT @SheeeRatchet: find someone who loves you a...,RT @SheeeRatchet: find someone who loves you a...,https://t.co/pbIoDo9bfy,NaN,NaN,NaN
